In [1]:
import os
import sys
import yaml
from copy import deepcopy

import torch
import torch.nn as nn

import numpy as np

sys.path.append('../../../')

%load_ext autoreload
%autoreload 2

from video_processing.yolov7.utils.general import make_divisible
from video_processing.yolov7.models.common import Conv, ReOrg, IAuxDetect, SPPCSPC, Concat

In [2]:
class Model(nn.Module):
    def __init__(self, cfg='yolor.yaml', ch=3,nc=None, anchors=None):
        super(Model, self).__init__()
        
    def forward(self):
        pass

In [3]:
cfg='yolov7-w6.yaml'

model=Model(cfg=cfg, ch=3, nc=80, anchors=None)

model.yaml_file=cfg
with open(cfg) as f: model.yaml=yaml.load(f, Loader=yaml.SafeLoader)

In [4]:
ch,nc=3,80
anchors=None
print('ch ', 'ch' in model.yaml, model.yaml.get('ch', ch), ch)
model.yaml['ch']=model.yaml.get('ch', ch)

#self.model, self.save = parse_model(deepcopy(self.yaml), ch=[ch])  # model, savelist
def parse_model(d, ch):
    '''
    d (dict): model configuration
    ch (list[int]): channel
    '''
    anchors, nc, depth_multiple, width_multiple=d['anchors'], d['nc'], d['depth_multiple'],d['width_multiple']
    n_anchors=(len(anchors[0])//2) if isinstance(anchors, list) else anchors # number of anchors
    n_outputs=n_anchors*(nc+5) # number of outputs = anchors*(num_classes+5)

    layers, save, c2=[],[], ch[-1] # layers, savelist, ch out
    all_n=[]
    for i, (f, n, m, args) in enumerate(d['backbone']+d['head']): # from, number, module, args
        print(i, '-'*100)
        print('args ', args)
        m=eval(m) if isinstance(m, str) else m # turn string to module from video_processing.yolov7.models.common
        # make sure that elements in args are int --> though some are None
        for j, a in enumerate(args):
            try: args[j]=eval(a) if isinstance(a,str) else a 
            except: pass
        print('args ', args) 
        n=max(round(n*depth_multiple), 1) if n>1 else n # depth gain
        all_n.append(n)
        print('n ', n)
        if m in [Conv, SPPCSPC]:
            ch_in, ch_out=ch[f], args[0]
            if ch_out!=n_outputs: ch_out=make_divisible(ch_out*width_multiple, 8)
            args=[ch_in, ch_out, *args[1:]]
            print(f'For Conv, SPPCSPC args: {args} from ch[f] {ch[f]} and f {f}')
            if m in [SPPCSPC]:
                args.insert(2, n) # number of repeats?
                print(f'For SPPCSPC n {n}, args {args}')
                n=1
        elif m is Concat:
            print(f'For Concat ch {ch} f {f}', end=', ')
            ch_out = sum([ch[x] for x in f])
            print(f'ch_out {ch_out}')
        elif m in [IAuxDetect]:
            print(f'For IAuxDetect ch {ch} f {f}')
            args.append([ch[x] for x in f])
            print(f'args {args}')
        elif m is ReOrg:
            print(f'For ReOrg ch {ch} f {f}', end=', ')
            ch_out = ch[f] * 4
            print(f' ch_out {ch_out}')
        else:
            print(f'For else ch {ch} f {f}', end=', ')
            ch_out=ch[f]
            print(f' ch_out {ch_out}')
        print(f'Final args {args} n {n}')
        m_=m(*args) # module
        print('m_ ', m_)
        t=str(m)
        t=t[t.rfind('.')+1:].strip("'>") #str(m)[8:-2].replace('__main__.','') # module type
        n_params=sum(x.numel() for x in m_.parameters()) # number of parameters
        m_.i, m_.f, m_.type, m_.np = i, f, t, n_params # index, from, type, n_params
        print(f'i {i} f {f}')
        save.extend(x%i for x in ([f] if isinstance(f, int) else f) if x!=-1) # append to save list
        print('t ', t,  ' np ', n_params)
        print('save ', save)
        layers.append(m_)
        if i==0: ch=[]
        ch.append(ch_out)
        print('ch ', ch)

    print('all_n ', np.unique(all_n))
    return nn.Sequential(*layers), sorted(save)
        
_, _=parse_model(deepcopy(model.yaml), ch=[ch])  # model, savelist

ch  False 3 3
0 ----------------------------------------------------------------------------------------------------
args  []
args  []
n  1
For ReOrg ch [3] f -1,  ch_out 12
Final args [] n 1
m_  ReOrg()
i 0 f -1
t  ReOrg  np  0
save  []
ch  [12]
1 ----------------------------------------------------------------------------------------------------
args  [64, 3, 1]
args  [64, 3, 1]
n  1
For Conv, SPPCSPC args: [12, 64, 3, 1] from ch[f] 12 and f -1
Final args [12, 64, 3, 1] n 1
m_  Conv(
  (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): SiLU()
)
i 1 f -1
t  Conv  np  7040
save  []
ch  [12, 64]
2 ----------------------------------------------------------------------------------------------------
args  [128, 3, 2]
args  [128, 3, 2]
n  1
For Conv, SPPCSPC args: [64, 128, 3, 2] from ch[f] 64 and f -1
Final args [64, 128, 3, 2] n 1
m_  Conv(
  (conv): C